In this file I am generating CMAs on EV charging stations buyt increasing the complexity or stealthiness of attack scenarios

In [1]:
import os
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
import networkx as nx
from  DSSCircuit.DSS_CircuitSetup import*

In [2]:
FolderName = os.getcwd()  # Gets the current directory where the notebook is running

In [3]:
mat_data = loadmat('./ChargingData/chargerdata_Wednesday.mat') #Reading the charging station data
# mat_data = loadmat('./ChargingData/chargerdata_Saturday.mat') #Reading the charging station data

In [4]:
charger_data = mat_data['chargerdata']

In [5]:
data_rows = []
DFWCharg_Select = []

In [6]:
# Loop through each EV charger entry
for charger_name in charger_data.dtype.names:
    daily_time = charger_data[charger_name][0,0]['daily_time'][0][0].flatten()
    daily_charging_power = charger_data[charger_name][0,0]['daily_charging_power'][0][0].flatten()
    daily_charging_current = charger_data[charger_name][0,0]['daily_charging_current'][0][0].flatten()
    daily_charging_voltage = charger_data[charger_name][0,0]['daily_charging_voltage'][0][0].flatten()
    latitude = charger_data[charger_name][0,0]['Latitude'][0][0].flatten()
    longitude = charger_data[charger_name][0,0]['Longitude'][0][0].flatten()
    data = {'Latitude':latitude, 
            'Longitude':longitude, 
            'daily_charging_current':daily_charging_current,
            'daily_charging_power':daily_charging_power,
            'daily_charging_voltage':daily_charging_voltage,
            'daily_time': daily_time}
    DFWCharg_Select.append({'name':charger_name, 'data':data})

In [7]:
for d in DFWCharg_Select:
    d['PBase'] = (max(d['data']['daily_charging_power'])*1.05)/1000 # slighlty more than maximum and converting it to kW

In [8]:
# Charging station Info 
# I verified the bus voltages 
StationsInfo =[{'no':1, 'bus':'814', 'numphase':3, 'phaseconn':'.1.2.3', 'kV':24.9, 'kw':DFWCharg_Select[0]['PBase'], 'kwh':770000, 'stored':0, 'reserve':0},
               {'no':2, 'bus':'890', 'numphase':3, 'phaseconn':'.1.2.3', 'kV':4.16, 'kw':DFWCharg_Select[1]['PBase'], 'kwh':480000, 'stored':0, 'reserve':0},
               {'no':3, 'bus':'852', 'numphase':3, 'phaseconn':'.1.2.3', 'kV':24.9, 'kw':DFWCharg_Select[2]['PBase'], 'kwh':1040000, 'stored':0, 'reserve':0}]

In [9]:
DFWCharg_Select

[{'name': 'EVcharger_336',
  'data': {'Latitude': array([30.706437]),
   'Longitude': array([-95.547224]),
   'daily_charging_current': array([170.71875, 170.71875, 170.71875, ...,   0.     ,   0.     ,
            0.     ]),
   'daily_charging_power': array([63278.93493652, 63291.53137207, 63304.12780762, ...,
              0.        ,     0.        ,     0.        ]),
   'daily_charging_voltage': array([370.66189236, 370.73567708, 370.80946181, ..., 416.17378818,
          416.17378818, 416.17378818]),
   'daily_time': array([0.00000000e+00, 1.00027785e+00, 2.00055571e+00, ...,
          8.63979994e+04, 8.63989997e+04, 8.64000000e+04])},
  'PBase': 73.6898722396172},
 {'name': 'EVcharger_442',
  'data': {'Latitude': array([30.010498]),
   'Longitude': array([-95.27120625]),
   'daily_charging_current': array([60.51375, 60.51375, 60.51375, ...,  0.     ,  0.     ,  0.     ]),
   'daily_charging_power': array([22430.14108887, 22434.6060791 , 22439.07106934, ...,
              0.       

In [10]:
# Load the OpenDSS circuit and Add charger station as input to OpenDSS circuit
#------------ Define the network with additions of charging stations -------------------------------------
def initialize():           
    DSSfile = os.path.join(FolderName, "DSSCircuit", "ieee34Mod1.DSS")  # Construct the path to your DSS file
    DSSCktobj = CktModSetup(DSSfile) 
    #----------------- Storage Allocation --------------------------------------------------
    for ChargingStation in StationsInfo:
        DSSCktobj.dss.Text.Command("New Storage.ChargeStatn" + str(ChargingStation['no']) + " bus1=" + ChargingStation['bus'] + ChargingStation['phaseconn'] + " phases=" + str(ChargingStation['numphase']) + " kv=" + str(ChargingStation['kV']) + " kW=" + str(ChargingStation['kw'])+ " kWrated=" + str(ChargingStation['kw']) +
                                   " kWhrated =" + str(ChargingStation['kwh']) + " %stored=" + str(ChargingStation['stored'])+ " %reserve=" + str(ChargingStation['reserve']) + " pf=1 dispmode=follow debugtrace=no Vminpu=0.1 Vmaxpu=10 model=1")
       
    DSSCktobj.dss.Solution.Solve() #solving snapshot power flows
    if DSSCktobj.dss.Solution.Converged():
        conv_flag=1
    else:
        conv_flag=0    
    G_init=graph_struct(DSSCktobj)
    return DSSCktobj,G_init,conv_flag

In [11]:
DSSCktobj,G_init,conv_flag= initialize() 

In [12]:
for i in range(len(StationsInfo)):
    elem = 'ChargeStatn'+str(StationsInfo[i]['no'])
    charge_P = -1*(DFWCharg_Select[i]['data']['daily_charging_power']/(StationsInfo[i]['kw']*1000)) # converting the charging power to kW, normalizing wrt to max Power and -ve (indicate charging)
    charge_time = DFWCharg_Select[i]['data']['daily_time'] # original time steps of charging data
    # Create a new shape and assign it to the storage device
    shape_name = "StorageShape"+str(StationsInfo[i]['no'])
    DSSCktobj.dss.LoadShape.New(shape_name)
    DSSCktobj.dss.LoadShape.Name(shape_name)
    DSSCktobj.dss.LoadShape.Npts(len(charge_P))     # Number of points
    DSSCktobj.dss.LoadShape.PMult(charge_P)         # Real power in kW (negative for charging)
    DSSCktobj.dss.LoadShape.SInterval(1)   #1 second interval
    DSSCktobj.dss.Text.Command(f'Edit Storage.{elem} Daily={shape_name}')
DSSCktobj.dss.Solution.Solve()

In [13]:
# Solve time series powerflow
Ttot = 24 # 24 hour siulations
DSSCktobj.dss.Text.Command('Set mode=daily') # daily simulation mode
DSSCktobj.dss.Text.Command('Set stepsize=5m') # time steps or resolution # This can be changed depending on desired resolution
# If you change time step here please also change when you plot.
DSSCktobj.dss.Text.Command('Set number=1') # The Solve command solves number of timesteps
t = 0

V_node_Pre = {}
states_Pre = {}
powers_Pre = {}
for s in StationsInfo:
    V_node_Pre[s['bus']]=[]
    states_Pre[s['bus']]=[]
    powers_Pre[s['bus']]=[]
while t<Ttot:
        DSSCktobj.dss.Solution.Solve()        
        i = DSSCktobj.dss.Storages.First()
        while i>0:
                bus = StationsInfo[i-1]['bus']
                states_Pre[bus].append(DSSCktobj.dss.Storages.State()) # Storage state
                powers_Pre[bus].append(sum(DSSCktobj.dss.CktElement.Powers()[::2])) # Circuit element power
                # Check Node Voltages            
                V=Bus(DSSCktobj,bus).Vmag
                V_node_Pre[bus].append(V)
                i = DSSCktobj.dss.Storages.Next()
        t  = int(DSSCktobj.dss.Solution.DblHour())

# Charging Attacks Generation

In [14]:
node_list = list(G_init.nodes())
edge_list = list(G_init.edges())
peak_start = 9
peak_end = 19
Scenarios = []
Tot_scenarios = 1000

In [15]:
for count in range(Tot_scenarios):
    DSSCktobj_Sc,_,conv_flag= initialize() #reset the circuit
    
    # Number of stations to be attacked selected
#     attack_no = random.randint(1,len(StationsInfo))# ALl the stations can be attacked 
    attack_no = 1
    statn_attack = random.sample(range(-1,len(StationsInfo)),attack_no) #Stations to be attacked are randomly picked
    statn_attack  = [x for x in statn_attack  if x != -1]
    if len(statn_attack) == 0:
        anomaly_indicator = 0
    else:
        anomaly_indicator = 1
    
    if anomaly_indicator==1:
        # The CMA types have been clubbed into 4 (1: type1, 2: type2&3, 3: type4&5, 4: type6)
        Types =  range(1,5)
        type_attack = random.choice(Types) # randomly select the type of the attack and all the stations in this scenario will be subjected to it    

        # Parameters randomly selected for different types of attacks and the attacks are coordinated so the same on different locations simultaneously
        attack_mult = random.uniform(1,5) # attack scaling factor for all types
        # for type 2 and 3 we need time shift 
        time_shift = random.choice(range(1,4)) #time shift duration
        shift_choice = random.choice([0,1]) #time shift direction 0: left(-ve), 1: right(+ve)
        if shift_choice == 0:
            time_shift = -1*time_shift 
        attack_mult2 = random.uniform(1,2) # attack scaling factor for type 6  
    else:
        type_attack = 0
        attack_mult = 1
        time_shift  = 0
        attack_mult2 = 1
    
    # Now imposing the attacks on the charging stations
    for i in range(len(StationsInfo)):
        charge_time = DFWCharg_Select[i]['data']['daily_time'] # original time steps of charging data
        charge_P=np.zeros(len(charge_time))
        elem = 'ChargeStatn'+str(StationsInfo[i]['no'])
        attack_max=StationsInfo[i]['kw']*attack_mult    
        if i in statn_attack: #if the attack is on the specific station
            # Type 1 
            if type_attack==1:
                for dindx in range(len(charge_time)): #for each index of charging profile
                    # If the charging time is within the attack duration (peak)
                        if (charge_time[dindx] > (peak_start*3600)) and (charge_time[dindx] < (peak_end*3600)):
                            charge_P[dindx] = -1*attack_mult*(DFWCharg_Select[i]['data']['daily_charging_power'][dindx]/(attack_max*1000)) # converting the charging power to kW, normalizing wrt to max Power and -ve (indicate charging)
                        else:
                            charge_P[dindx]=-1*(DFWCharg_Select[i]['data']['daily_charging_power'][dindx]/(attack_max*1000))   
            
            # Type 2 (Shifting peak to right or left and increasing it)
            elif type_attack==2:
                for dindx in range(len(charge_time)): #for each index of charging profile
                    charge_P[dindx]=-1*(DFWCharg_Select[i]['data']['daily_charging_power'][dindx]/(attack_max*1000))  
                # Shifting the charging demand to the left or right
                charge_P[((peak_start+time_shift)*3600):((peak_end+time_shift)*3600)] = attack_mult*charge_P[(peak_start*3600):(peak_end*3600)]    

            # Type 3 (Increasing demand and then shifting)
            elif type_attack==3:   
                for dindx in range(len(charge_time)): #for each index of charging profile
                    # If the charging time is within the attack duration (peak)
                        if (charge_time[dindx] > (peak_start*3600)) and (charge_time[dindx] < (peak_end*3600)):
                            charge_P[dindx] = -1*attack_mult*(DFWCharg_Select[i]['data']['daily_charging_power'][dindx]/(attack_max*1000)) # converting the charging power to kW, normalizing wrt to max Power and -ve (indicate charging)
                        else:
                            charge_P[dindx]=-1*(DFWCharg_Select[i]['data']['daily_charging_power'][dindx]/(attack_max*1000))                                                                                               
                    # Shifting the charging demand to the right
                charge_P[((peak_start+time_shift)*3600):((peak_end+time_shift)*3600)] = charge_P[(peak_start*3600):(peak_end*3600)] 

            #Type 4
            elif type_attack==4:
                for dindx in range(len(charge_time)): #for each index of charging profile
                    # If the charging time is within the attack duration (peak)
                        if (charge_time[dindx] > (peak_start*3600)) and (charge_time[dindx] < (peak_end*3600)):
                            charge_P[dindx] = -1*attack_mult*(DFWCharg_Select[i]['data']['daily_charging_power'][dindx]/(attack_max*1000)) # converting the charging power to kW, normalizing wrt to max Power and -ve (indicate charging)
                        else:
                            charge_P[dindx]=-1*(DFWCharg_Select[i]['data']['daily_charging_power'][dindx]/(attack_max*1000))                                                                                               
                    # Shifting the charging demand to the right
                charge_P[((peak_start+time_shift)*3600):((peak_end+time_shift)*3600)] = attack_mult2*charge_P[(peak_start*3600):(peak_end*3600)] 
        
            # Change the ratings of station if its attacked to prevent clipping by openDSS (after KWrated(1 pu))
            DSSCktobj_Sc.dss.Text.Command(f'Edit Storage.{elem} kW={attack_max} kWrated={attack_max} model=1 Vminpu=0.1 Vmaxpu=10')
        
        # If station not attacked
        else:
            charge_P = -1*(DFWCharg_Select[i]['data']['daily_charging_power']/(StationsInfo[i]['kw']*1000)) # converting the charging power to kW, normalizing wrt to max Power and -ve (indicate charging)
            
        # Create a new shape and assign it to the storage device for the station
        shape_name = "StorageShape"+str(StationsInfo[i]['no'])
        DSSCktobj_Sc.dss.LoadShape.New(shape_name)
        DSSCktobj_Sc.dss.LoadShape.Name(shape_name)
        DSSCktobj_Sc.dss.LoadShape.Npts(len(charge_P))     # Number of points
        DSSCktobj_Sc.dss.LoadShape.PMult(charge_P)         # Real power in kW (negative for charging)
        DSSCktobj_Sc.dss.LoadShape.SInterval(1)   # 1 second interval
        DSSCktobj_Sc.dss.Text.Command(f'Edit Storage.{elem} Daily={shape_name}')
    DSSCktobj_Sc.dss.Solution.Solve() 

    # Solve time series powerflow
    Ttot = 24 # 24 hour siulations
    DSSCktobj_Sc.dss.Text.Command('Set mode=daily') # daily simulation mode
    DSSCktobj_Sc.dss.Text.Command('Set stepsize=5m') # resolution or time steps # This can be changed depending on desired resolution
    # If you change time step here please also change when you plot.
    DSSCktobj_Sc.dss.Text.Command('Set number=1') # The Solve command solves number of timesteps
    t = 0

    V_node_Sc = {bus: [] for bus in node_list}
    flow_branch_Sc= {br: [] for br in edge_list}
    powers_Sc = {station['bus']:[] for station in StationsInfo}
    while t<Ttot:
            DSSCktobj_Sc.dss.Solution.Solve()        
            # Get Node Voltages
            for bus in node_list:
                V=Bus(DSSCktobj_Sc,bus).Vmag
                V_node_Sc[bus].append(V)
            # Get branchflows  
            for (u,v) in edge_list:
                Power_br=0
                branchelems = G_init[u][v]['label']
                for b in branchelems:
                    branchflow = Branch(DSSCktobj_Sc,b).flow
                    Power_br =Power_br + sum(branchflow)
                flow_branch_Sc[(u,v)].append(Power_br)
            # Get storage powers  
            i = DSSCktobj_Sc.dss.Storages.First()
            while i>0:
                    sbus = StationsInfo[i-1]['bus']
                    powers_Sc[sbus].append(sum(DSSCktobj_Sc.dss.CktElement.Powers()[::2])) # Circuit element power
                    i = DSSCktobj_Sc.dss.Storages.Next()

            t  = int(DSSCktobj_Sc.dss.Solution.DblHour())      
    Scenario_Info ={'Anomaly':anomaly_indicator, 'Stations Attacked':statn_attack, 'Type':type_attack, 'Bus Voltages':  V_node_Sc, 'Branch flows':flow_branch_Sc, 'Charging Powers':powers_Sc} 
    Scenarios.append(Scenario_Info)

In [16]:
with open(FolderName+ './WeekdayScenarios.pkl', 'wb') as file:
    pickle.dump(Scenarios, file)

In [17]:
len(Scenarios)

1000